In [11]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [30]:
a_true, b_true = 1, 1
True_value =[a_true, b_true]
x = np.linspace(0.1,1,10)
def mean(theta):
    a,b = theta
    return np.cos(a*x**2 + b*x)
y_mean = mean([a_true, b_true])
std = .01*abs(max(y_mean))

In [31]:
y_random = np.random.randn(200, len(x)) 

In [32]:
y_sample = y_random * std + y_mean
cov = np.cov(y_sample.T)

In [33]:
import iminuit
from iminuit import Minuit
def lnL(a, b):
    theta = a, b
    y_th = mean(theta)
    inv_cov = np.linalg.inv(cov)
    d = y_sample[0] - y_th
    res = -0.5*np.sum(d*inv_cov.dot(d))
    return res
def lnL_emcee(theta):
    a, b = theta
    return lnL(a, b)
def neg_lnL_minuit(a, b):
    return -lnL(a, b)

In [34]:
minuit = Minuit(neg_lnL_minuit, a = a_true, b= b_true,
                   errordef = 0.5)  
minuit.migrad(),minuit.hesse(),minuit.minos()
cov_minuit = np.array(minuit.matrix())
MLE = [minuit.values['a'], minuit.values['b']]

In [35]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/pbs/throng/lsst/users/cpayerne/GitForThesis/DC2Analysis/modeling/')
import DATAOPS_Fisher_matrix as fisher
Fisher_matrix = fisher.Fisher_Matrix(True_value, mean, cov, delta = 1e-5)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
cov_Fisher = np.linalg.inv(Fisher_matrix)

In [37]:
cov_Fisher

array([[ 0.00102526, -0.00086985],
       [-0.00086985,  0.00077066]])

In [38]:
cov_minuit

array([[ 0.0010318 , -0.00087524],
       [-0.00087524,  0.00077516]])